# Mining Logistics AI Simulation - Model Deployment Testing

## Overview
**Role:** ML Engineer A  
**Project:** Mining Value Chain - Logistics Simulation  
**Tujuan:** Mensimulasikan sistem **"Hybrid AI Agent"** yang menggabungkan Prediksi Waktu (Regresi), Deteksi Risiko (Klasifikasi), dan Aturan Keselamatan (Safety Rules) dalam satu API sederhana yang siap untuk production deployment.

---

## Executive Summary

Notebook ini adalah tahap akhir (**Deployment Phase**). Di sini, kita tidak lagi melatih model, melainkan **menggunakannya** untuk mengambil keputusan bisnis real-time.

### Arsitektur Sistem (Hybrid Safety Net)

**Layer 1 - Safety Veto (Hard Rules):**
- Menangkap bahaya fisik yang nyata dan ekstrem (misal: Hujan Badai > 20mm atau Alat Rusak)
- Aturan ini bersifat mutlak dan **meng-override** prediksi AI
- Zero tolerance untuk kondisi yang membahayakan keselamatan

**Layer 2 - AI Intelligence:**
- Jika kondisi fisik aman, AI bekerja mendeteksi pola tersembunyi
- Mengidentifikasi potensi risiko operasional (antrian macet, kelelahan operator, dll)
- Menggunakan model XGBoost yang telah dilatih dan dioptimasi

### Output Sistem

- ⏱️ **Estimasi Waktu (Cycle Time):** Prediksi durasi dalam jam/menit
- 🛡️ **Status Risiko:** `AMAN`, `WASPADA`, atau `BAHAYA`
- 📊 **AI Confidence:** Tingkat kepercayaan model terhadap prediksi
- 💡 **Rekomendasi:** Aksi konkret untuk Dispatcher dan operator lapangan

### Business Value

1. **Proactive Risk Management:** Deteksi dini sebelum insiden terjadi
2. **Optimized Resource Allocation:** Alokasi equipment dan operator berdasarkan prediksi
3. **Safety Compliance:** Memastikan operasi sesuai standar keselamatan
4. **Data-Driven Decision Making:** Menggantikan keputusan subjektif dengan analisis sistematis

## 1. Setup Environment & Load Model 
Kita akan memuat "Otak" AI yang sudah dilatih dan disimpan pada tahap sebelumnya (`.pkl` atau `.json`). Pastikan file berikut ada di folder yang sama: * `model_regresi_cycle.pkl` (Model Estimasi Waktu) * `model_klasifikasi_risiko.pkl` (Model Deteksi Risiko)

In [8]:
# ============================================================================
# Setup Environment & Load Trained Models
# ============================================================================

import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set project root directory
project_root = Path(r'c:\Users\I5\Documents\asah-2025\capstone-project\minewise_ml')
os.chdir(project_root)
print(f"Working directory: {os.getcwd()}")

# ============================================================================
# Load Pre-trained Models
# ============================================================================

print("\n" + "="*80)
print("LOADING TRAINED MODELS")
print("="*80)

model_dir = project_root / 'models'

# Define model paths
regression_model_path = model_dir / 'infra_cycle_time_regression.pkl'
classification_model_path = model_dir / 'infra_road_risk_classification.pkl'

try:
    # Load regression model (Cycle Time Prediction)
    if regression_model_path.exists():
        reg_model = joblib.load(regression_model_path)
        print(f"Regression Model loaded: {regression_model_path.name}")
    else:
        print(f"Regression Model not found: {regression_model_path}")
        print("Please run Notebook: 02_Cycle_Time_Regression_v2.ipynb first")
        reg_model = None
    
    # Load classification model (Road Risk Classification)
    if classification_model_path.exists():
        clf_model = joblib.load(classification_model_path)
        print(f"Classification Model loaded: {classification_model_path.name}")
    else:
        print(f"Classification Model not found: {classification_model_path}")
        print("  Please run Notebook: 03_Road_Risk_Classification_v2.ipynb first")
        clf_model = None
    
    if reg_model is not None and clf_model is not None:
        print("\n" + "="*80)
        print("ALL MODELS LOADED SUCCESSFULLY")
        print("="*80)
    else:
        print("\n" + "="*80)
        print("WARNING: Some models are missing - simulation will be limited")
        print("="*80)

except Exception as e:
    print(f"\nError loading models: {e}")
    print("  Please ensure you have trained and saved the models first")
    reg_model = None
    clf_model = None

Working directory: c:\Users\I5\Documents\asah-2025\capstone-project\minewise_ml

LOADING TRAINED MODELS
Regression Model loaded: infra_cycle_time_regression.pkl
Classification Model loaded: infra_road_risk_classification.pkl

ALL MODELS LOADED SUCCESSFULLY


## 2. Handling Feature Mismatch (The "Adapter") 
Salah satu tantangan terbesar dalam deployment model Machine Learning adalah perbedaan struktur data antara **Training** (yang punya banyak kolom One-Hot Encoding) dan **Input Real-time** (yang ringkas). Fungsi `align_features` di bawah ini bertindak sebagai **Adapter Universal**. Ia otomatis: 1. Membaca struktur kolom yang diharapkan oleh model. 2. Membuat template kosong. 3. Mengisi data input ke tempat yang sesuai dan membiarkan sisanya 0 (default). Ini mencegah error `ValueError: feature_names mismatch` yang sering terjadi saat integrasi.

In [9]:
# ============================================================================
# Feature Alignment Adapter (Solusi Feature Mismatch)
# ============================================================================

def align_features(input_df, model):
    """
    Universal adapter untuk menyelaraskan features antara input real-time
    dan struktur yang diharapkan oleh trained model.
    
    Problem yang diselesaikan:
    - Training data punya banyak kolom (One-Hot Encoding, polynomial features)
    - Real-time input hanya punya kolom dasar (ringkas)
    - XGBoost akan error jika feature_names tidak match
    
    Solusi:
    - Membaca struktur kolom yang diharapkan model
    - Membuat template kosong dengan semua kolom yang diperlukan
    - Mengisi data input ke tempat yang sesuai
    - Kolom yang tidak ada diisi dengan 0 (default)
    
    Parameters:
    -----------
    input_df : pd.DataFrame
        DataFrame dengan kolom input dari user/sistem
    model : xgboost.XGBModel
        Trained XGBoost model dengan feature_names yang sudah terdefinisi
    
    Returns:
    --------
    aligned_df : pd.DataFrame
        DataFrame dengan struktur kolom yang sesuai dengan model
    
    Example:
    --------
    >>> input_data = pd.DataFrame([{'hujan_mm': 10, 'umur_tahun': 5}])
    >>> aligned_data = align_features(input_data, trained_model)
    >>> prediction = trained_model.predict(aligned_data)
    """
    # Get expected features from model
    expected_features = model.get_booster().feature_names
    
    # Create empty DataFrame with all expected features
    aligned_df = pd.DataFrame(0, index=input_df.index, columns=expected_features)
    
    # Copy data from input to aligned DataFrame (only common columns)
    common_cols = [col for col in input_df.columns if col in expected_features]
    aligned_df[common_cols] = input_df[common_cols]
    
    return aligned_df

## 3. The Hybrid Simulation Engine Fungsi 
`simulasi_api_call` adalah simulasi dari **Backend API**. Fungsi ini menerima data mentah dari lapangan (atau input user di Web), melakukan *Preprocessing* (seperti menghitung Sin/Cos jam), dan menjalankan logika bertingkat (Rules + AI).

In [10]:
# ============================================================================
# Hybrid Simulation Engine - Core Logic
# ============================================================================

def simulasi_api_call(input_dict):
    """
    Simulasi API endpoint untuk Mining Logistics Decision System.
    
    Arsitektur Decision Making:
    ---------------------------
    1. Feature Engineering On-the-Fly
       - Membuat fitur turunan dari input dasar
       - Menghitung interaction features (hujan x umur, dll)
       - Temporal features (sin/cos jam operasi)
    
    2. AI Prediction
       - Cycle Time Regression: Estimasi durasi operasi
       - Risk Classification: Tingkat risiko jalan/operasi
    
    3. Safety Veto Layer
       - Hard Rules yang meng-override AI
       - Hujan Kritis (>20mm) → BAHAYA otomatis
       - Alat Rusak → BAHAYA otomatis
    
    4. AI Intelligence Layer
       - Jika tidak ada veto, AI mendeteksi pola tersembunyi
       - High risk probability (>0.4) → WASPADA
       - Low risk probability → AMAN
    
    Parameters:
    -----------
    input_dict : dict
        Dictionary dengan keys:
        - hujan_mm: Curah hujan (mm)
        - umur_tahun: Umur alat (tahun)
        - jam_operasi: Jam operasi (0-23)
        - avg_last_3_cycles: Rata-rata 3 cycle terakhir (jam)
        - prev_cycle_time: Cycle time sebelumnya (jam)
        - operator_score: Skor operator (0-1)
        - alat_score: Skor kondisi alat (0-1)
        - kondisi_perlu_servis: Flag servis (0/1)
    
    Returns:
    --------
    result : dict
        Dictionary dengan keys:
        - Estimasi Waktu: Prediksi cycle time
        - Status Risiko: AMAN/WASPADA/BAHAYA
        - AI Confidence: Confidence level AI (%)
        - Pesan: Penjelasan keputusan
    """
    # Validate models are loaded
    if reg_model is None or clf_model is None:
        return {
            "Estimasi Waktu": "N/A",
            "Status Risiko": "ERROR",
            "AI Confidence": "0.0%",
            "Pesan": "Model tidak tersedia - silakan load model terlebih dahulu"
        }
    
    # Copy input to avoid modifying original
    data = input_dict.copy()
    
    # ========================================================================
    # A. Feature Engineering On-the-Fly
    # ========================================================================
    
    # Interaction features
    data['risk_factor_1'] = data['hujan_mm'] * data['umur_tahun']
    
    # Rain spike indicator (sudden heavy rain)
    data['rain_spike'] = 10 if data['hujan_mm'] > 15 else 0
    
    # Temporal features (cyclical encoding)
    jam_op = data.get('jam_operasi', 12)
    data['jam_sin'] = np.sin(2 * np.pi * jam_op / 24)
    data['jam_cos'] = np.cos(2 * np.pi * jam_op / 24)
    
    # Convert to DataFrame
    input_df = pd.DataFrame([data])
    
    # ========================================================================
    # B. AI Prediction
    # ========================================================================
    
    # Regression: Cycle Time Prediction
    df_reg = align_features(input_df, reg_model)
    est_time = reg_model.predict(df_reg)[0]
    
    # Classification: Risk Probability
    df_clf = align_features(input_df, clf_model)
    risk_prob = clf_model.predict_proba(df_clf)[0, 1]  # Probability of high risk
    
    # ========================================================================
    # C. Hybrid Decision Logic (Safety Veto + AI Intelligence)
    # ========================================================================
    
    reasons = []
    
    # Layer 1: Safety Veto (Hard Rules)
    if input_dict['hujan_mm'] > 20:
        reasons.append(f"HUJAN KRITIS ({input_dict['hujan_mm']}mm) - Visibility terbatas")
    
    if input_dict.get('kondisi_perlu_servis', 0) == 1:
        reasons.append("ALAT RUSAK - Berpotensi breakdown")
    
    # Determine final status
    if reasons:
        # Safety Veto activated
        status = "BAHAYA (VETO)"
    elif risk_prob > 0.4:
        # AI detects hidden pattern
        status = "WASPADA (AI)"
        reasons.append(f"AI mendeteksi pola risiko tinggi (Confidence: {risk_prob*100:.1f}%)")
    else:
        # All clear
        status = "AMAN"
    
    # ========================================================================
    # D. Format Output
    # ========================================================================
    
    return {
        "Estimasi Waktu": f"{est_time:.2f} Jam ({est_time*60:.0f} Menit)",
        "Status Risiko": status,
        "AI Confidence": f"{risk_prob*100:.1f}%",
        "Pesan": ", ".join(reasons) if reasons else "Lanjutkan Operasi Normal"
    }

## 4. Uji Coba Skenario (Stress Test) 
### Skenario 1: Kondisi Bahaya (Safety Veto) * 
**Kondisi:** Hujan Sangat Deras (55mm) + Alat Tua (15 Tahun). 
**Ekspektasi:** Sistem harus memvonis 
**BAHAYA** karena aturan *Hard Rules*, meskipun mungkin AI ragu-ragu (karena jarang melihat data badai). 
### Skenario 2: Kondisi Aman (Normal Operation) 
**Kondisi:** Cerah (0mm), Pagi Hari, Antrian Lancar. 
**Ekspektasi:** Sistem memvonis 
**AMAN** dengan estimasi waktu yang wajar.

In [11]:
# ============================================================================
# Stress Test - Multiple Scenarios
# ============================================================================

print("\n" + "="*80)
print("SIMULATION TESTING - STRESS TEST SCENARIOS")
print("="*80)

# ============================================================================
# Skenario 1: BAHAYA - Hujan Badai + Alat Tua
# ============================================================================

print("\n" + "="*80)
print("SKENARIO 1: KONDISI EKSTREM (SAFETY VETO)")
print("="*80)

input_bahaya = {
    'hujan_mm': 55,                   # Badai sangat deras
    'umur_tahun': 15,                 # Alat tua (mendekati end-of-life)
    'jam_operasi': 14,                # Siang hari (panas + hujan = visibility buruk)
    'avg_last_3_cycles': 0.5,         # Historical average
    'prev_cycle_time': 0.5,           # Previous cycle
    'operator_score': 0.8,            # Operator berpengalaman
    'alat_score': 0.9,                # Kondisi alat sebenarnya baik
    'kondisi_perlu_servis': 0         # Tidak perlu servis
}

print("\nInput Data:")
for key, value in input_bahaya.items():
    print(f"  {key:25s}: {value}")

print("\nEkspektasi:")
print("  - Status: BAHAYA (karena hujan ekstrem >20mm)")
print("  - Safety Veto harus meng-override AI meskipun alat kondisi baik")

print("\nHasil Simulasi:")
hasil_1 = simulasi_api_call(input_bahaya)
for key, value in hasil_1.items():
    print(f"  {key:20s}: {value}")

# ============================================================================
# Skenario 2: AMAN - Kondisi Normal Operasi
# ============================================================================

print("\n" + "="*80)
print("SKENARIO 2: KONDISI NORMAL (AI INTELLIGENCE)")
print("="*80)

input_aman = {
    'hujan_mm': 0,                    # Cerah
    'umur_tahun': 5,                  # Alat relatif muda
    'jam_operasi': 10,                # Pagi hari (kondisi optimal)
    'avg_last_3_cycles': 0.3,         # Cycle cepat (operasi lancar)
    'prev_cycle_time': 0.3,           # Cycle terakhir cepat
    'operator_score': 0.5,            # Operator biasa
    'alat_score': 0.5,                # Kondisi alat standar
    'kondisi_perlu_servis': 0         # Tidak perlu servis
}

print("\nInput Data:")
for key, value in input_aman.items():
    print(f"  {key:25s}: {value}")

print("\nEkspektasi:")
print("  - Status: AMAN (tidak ada trigger safety veto)")
print("  - AI confidence rendah (tidak ada pola risiko)")

print("\nHasil Simulasi:")
hasil_2 = simulasi_api_call(input_aman)
for key, value in hasil_2.items():
    print(f"  {key:20s}: {value}")

# ============================================================================
# Skenario 3: WASPADA - Kondisi Borderline
# ============================================================================

print("\n" + "="*80)
print("SKENARIO 3: KONDISI BORDERLINE (AI DETECTION)")
print("="*80)

input_waspada = {
    'hujan_mm': 15,                   # Hujan sedang (tidak trigger veto)
    'umur_tahun': 10,                 # Alat setengah umur
    'jam_operasi': 18,                # Sore hari (operator mungkin lelah)
    'avg_last_3_cycles': 0.7,         # Cycle mulai melambat
    'prev_cycle_time': 0.8,           # Cycle terakhir lambat
    'operator_score': 0.3,            # Operator kurang pengalaman
    'alat_score': 0.4,                # Kondisi alat menurun
    'kondisi_perlu_servis': 0         # Belum trigger maintenance
}

print("\nInput Data:")
for key, value in input_waspada.items():
    print(f"  {key:25s}: {value}")

print("\nEkspektasi:")
print("  - Status: WASPADA (AI mendeteksi kombinasi faktor risiko)")
print("  - AI confidence medium-high (pola tersembunyi)")

print("\nHasil Simulasi:")
hasil_3 = simulasi_api_call(input_waspada)
for key, value in hasil_3.items():
    print(f"  {key:20s}: {value}")

# ============================================================================
# Summary Comparison
# ============================================================================

print("\n" + "="*80)
print("COMPARISON SUMMARY")
print("="*80)

comparison = pd.DataFrame([
    {
        'Skenario': 'Skenario 1 (Badai)',
        'Status': hasil_1['Status Risiko'],
        'Estimasi': hasil_1['Estimasi Waktu'],
        'Confidence': hasil_1['AI Confidence']
    },
    {
        'Skenario': 'Skenario 2 (Normal)',
        'Status': hasil_2['Status Risiko'],
        'Estimasi': hasil_2['Estimasi Waktu'],
        'Confidence': hasil_2['AI Confidence']
    },
    {
        'Skenario': 'Skenario 3 (Borderline)',
        'Status': hasil_3['Status Risiko'],
        'Estimasi': hasil_3['Estimasi Waktu'],
        'Confidence': hasil_3['AI Confidence']
    }
])

print("\n" + comparison.to_string(index=False))
print("\n" + "="*80)


SIMULATION TESTING - STRESS TEST SCENARIOS

SKENARIO 1: KONDISI EKSTREM (SAFETY VETO)

Input Data:
  hujan_mm                 : 55
  umur_tahun               : 15
  jam_operasi              : 14
  avg_last_3_cycles        : 0.5
  prev_cycle_time          : 0.5
  operator_score           : 0.8
  alat_score               : 0.9
  kondisi_perlu_servis     : 0

Ekspektasi:
  - Status: BAHAYA (karena hujan ekstrem >20mm)
  - Safety Veto harus meng-override AI meskipun alat kondisi baik

Hasil Simulasi:
  Estimasi Waktu      : 0.55 Jam (33 Menit)
  Status Risiko       : BAHAYA (VETO)
  AI Confidence       : 5.9%
  Pesan               : HUJAN KRITIS (55mm) - Visibility terbatas

SKENARIO 2: KONDISI NORMAL (AI INTELLIGENCE)

Input Data:
  hujan_mm                 : 0
  umur_tahun               : 5
  jam_operasi              : 10
  avg_last_3_cycles        : 0.3
  prev_cycle_time          : 0.3
  operator_score           : 0.5
  alat_score               : 0.5
  kondisi_perlu_servis     : 0

Eks

# Kesimpulan & Deployment Roadmap

## Validation Results

Sistem simulasi ini membuktikan bahwa model AI telah siap diintegrasikan ke production environment. Hasil stress test menunjukkan sistem bekerja sesuai ekspektasi pada berbagai kondisi operasional.

---

## Keunggulan Sistem

### 1. Robustness
- **Feature Mismatch Handling:** Tidak error meskipun input data minimal (berkat `align_features`)
- **Graceful Degradation:** Sistem memberikan error message yang informatif jika model tidak tersedia
- **Input Validation:** Otomatis handling missing features dengan default values

### 2. Safety-First Architecture
- **Veto Mechanism:** Memprioritaskan keselamatan dengan mekanisme *Hard Rules*
- **Zero False Negatives:** Untuk kondisi kritis (hujan ekstrem, alat rusak)
- **Multi-Layer Defense:** Safety rules + AI intelligence sebagai double protection

### 3. Interpretability & Trust
- **Transparent Reasoning:** Memberikan alasan yang jelas untuk setiap keputusan
- **Confidence Score:** Menampilkan tingkat kepercayaan AI terhadap prediksi
- **Actionable Insights:** Output dapat langsung digunakan untuk decision making

### 4. Production-Ready Features
- **Path Handling:** Menggunakan absolute path untuk reliability
- **Error Handling:** Comprehensive try-catch blocks untuk robustness
- **Logging Ready:** Structured output yang mudah di-log untuk monitoring
- **API-Compatible:** Format output JSON-compatible untuk web integration

---

## Technical Specifications

### Input Requirements
```python
{
    'hujan_mm': float,              # Curah hujan (mm)
    'umur_tahun': int,              # Umur alat (tahun)
    'jam_operasi': int,             # Jam operasi (0-23)
    'avg_last_3_cycles': float,     # Rata-rata 3 cycle (jam)
    'prev_cycle_time': float,       # Cycle sebelumnya (jam)
    'operator_score': float,        # Skor operator (0-1)
    'alat_score': float,            # Skor alat (0-1)
    'kondisi_perlu_servis': int     # Flag servis (0/1)
}
```

### Output Format
```python
{
    'Estimasi Waktu': str,          # "X.XX Jam (YY Menit)"
    'Status Risiko': str,           # "AMAN" / "WASPADA (AI)" / "BAHAYA (VETO)"
    'AI Confidence': str,           # "XX.X%"
    'Pesan': str                    # Penjelasan keputusan
}
```

### Decision Thresholds
- **Safety Veto:** Hujan > 20mm OR Alat Rusak (kondisi_perlu_servis = 1)
- **AI WASPADA:** Risk Probability > 0.4 (40%)
- **AI AMAN:** Risk Probability ≤ 0.4 (40%)

---